In [6]:
#import psycopg2
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, text, TIMESTAMP, func, DECIMAL, ForeignKey, CheckConstraint
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import os
from pathlib import Path

In [7]:
# Parámetros de conexión
host = "localhost"
port = "5432"
dbname = "EcommerceDB"
user = "postgres"
password = "postgres123"

# Crear conexión
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}", echo=True)
Session = sessionmaker(bind=engine)
session = Session()

In [9]:
Base = declarative_base()

class Usuario(Base):
    __tablename__ = 'usuarios'
    usuarioid = Column(Integer, primary_key=True, autoincrement=True)
    nombre = Column(String(100), nullable=False)
    apellido = Column(String(100), nullable=False)
    dni = Column(String(20), unique=True, nullable=False)
    email = Column(String(255), unique=True, nullable=False)
    contraseña = Column(String(255), nullable=False)
    fecharegistro = Column(TIMESTAMP, server_default=func.current_timestamp())

class Categoria(Base):
    __tablename__ = 'categorias'
    categoriaid = Column(Integer, primary_key=True, autoincrement=True)
    nombre = Column(String(100), nullable=False)
    descripcion = Column(String(255))

class Producto(Base):
    __tablename__ = 'productos'
    productoid = Column(Integer, primary_key=True, autoincrement=True)
    nombre = Column(String(255), nullable=False)
    descripcion = Column(String(255))
    precio = Column(DECIMAL(10, 2), nullable=False)
    stock = Column(Integer, nullable=False)
    categoriaid = Column(Integer, ForeignKey('categorias.categoriaid'), nullable=False)

class Orden(Base):
    __tablename__ = 'ordenes'
    ordenid = Column(Integer, primary_key=True, autoincrement=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'), nullable=False)
    fechaorden = Column(TIMESTAMP, server_default=func.current_timestamp())
    total = Column(DECIMAL(10, 2), nullable=False)
    estado = Column(String(50), server_default='Pendiente')

class DetalleOrden(Base):
    __tablename__ = 'detalleordenes'
    detalleid = Column(Integer, primary_key=True, autoincrement=True)
    ordenid = Column(Integer, ForeignKey('ordenes.ordenid'), nullable=False)
    productoid = Column(Integer, ForeignKey('productos.productoid'), nullable=False)
    cantidad = Column(Integer, nullable=False)
    preciounitario = Column(DECIMAL(10, 2), nullable=False)

class DireccionEnvio(Base):
    __tablename__ = 'direccionesenvio'
    direccionid = Column(Integer, primary_key=True, autoincrement=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'), nullable=False)
    calle = Column(String(255), nullable=False)
    ciudad = Column(String(100), nullable=False)
    departamento = Column(String(100))
    provincia = Column(String(100))
    distrito = Column(String(100))
    estado = Column(String(100))
    codigopostal = Column(String(20))
    pais = Column(String(100), nullable=False)

class Carrito(Base):
    __tablename__ = 'carrito'
    carritoid = Column(Integer, primary_key=True, autoincrement=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'), nullable=False)
    productoid = Column(Integer, ForeignKey('productos.productoid'), nullable=False)
    cantidad = Column(Integer, nullable=False)
    fechaagregado = Column(TIMESTAMP, server_default=func.current_timestamp())

class MetodoPago(Base):
    __tablename__ = 'metodospago'
    metodopagoid = Column(Integer, primary_key=True, autoincrement=True)
    nombre = Column(String(100), nullable=False)
    descripcion = Column(String(255))

class OrdenMetodoPago(Base):
    __tablename__ = 'ordenesmetodospago'
    ordenmetodoid = Column(Integer, primary_key=True, autoincrement=True)
    ordenid = Column(Integer, ForeignKey('ordenes.ordenid'), nullable=False)
    metodopagoid = Column(Integer, ForeignKey('metodospago.metodopagoid'), nullable=False)
    montopagado = Column(DECIMAL(10, 2), nullable=False)

class ReseñaProducto(Base):
    __tablename__ = 'reseñasproductos'
    reseñaid = Column(Integer, primary_key=True, autoincrement=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'), nullable=False)
    productoid = Column(Integer, ForeignKey('productos.productoid'), nullable=False)
    calificacion = Column(Integer, CheckConstraint('calificacion >= 1 AND calificacion <= 5'))
    comentario = Column(String(255))
    fecha = Column(TIMESTAMP, server_default=func.current_timestamp())

class HistorialPago(Base):
    __tablename__ = 'historialpagos'
    pagoid = Column(Integer, primary_key=True, autoincrement=True)
    ordenid = Column(Integer, ForeignKey('ordenes.ordenid'), nullable=False)
    metodopagoid = Column(Integer, ForeignKey('metodospago.metodopagoid'), nullable=False)
    monto = Column(DECIMAL(10, 2), nullable=False)
    fechapago = Column(TIMESTAMP, server_default=func.current_timestamp())
    estadopago = Column(String(50), server_default='Procesando')

Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)


2025-07-24 13:36:43,715 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-24 13:36:43,718 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-07-24 13:36:43,721 INFO sqlalchemy.engine.Engine [cached since 21.43s ago] {'table_name': 'usuarios', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-07-24 13:36:43,739 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.p

C:\Users\gabriel.b.alvarez\AppData\Local\Temp\ipykernel_20552\2515579819.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


2025-07-24 13:36:43,850 INFO sqlalchemy.engine.Engine 
DROP TABLE metodospago
2025-07-24 13:36:43,850 INFO sqlalchemy.engine.Engine [no key 0.00084s] {}
2025-07-24 13:36:43,855 INFO sqlalchemy.engine.Engine 
DROP TABLE categorias
2025-07-24 13:36:43,856 INFO sqlalchemy.engine.Engine [no key 0.00114s] {}
2025-07-24 13:36:43,860 INFO sqlalchemy.engine.Engine 
DROP TABLE usuarios
2025-07-24 13:36:43,863 INFO sqlalchemy.engine.Engine [no key 0.00229s] {}
2025-07-24 13:36:43,865 INFO sqlalchemy.engine.Engine COMMIT
2025-07-24 13:36:43,878 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-24 13:36:43,879 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_ca

In [15]:
#ruta_base = r"resources\csv"
ruta_base = Path.cwd().parents[1] / "resources" / "csv"

map_csv_table = {
    "2.Usuarios.csv": "usuarios",
    "3.Categorias.csv": "categorias",
    "4.Productos.csv": "productos",
    "5.ordenes.csv": "ordenes",
    "6.detalle_ordenes.csv": "detalleordenes",
    "7.direcciones_envio.csv": "direccionesenvio",
    "8.carrito.csv": "carrito",
    "9.metodos_pago.csv": "metodospago",
    "10.ordenes_metodospago.csv": "ordenesmetodospago",
    "11.resenas_productos.csv": "reseñasproductos",
    "12.historial_pagos.csv": "historialpagos"
}

for file_name, table_name in map_csv_table.items():
    #file_path = os.path.join(ruta_base, file_name)
    file_path = ruta_base / file_name
    df = pd.read_csv(file_path)

    # Proceso de Wrangling:
    # Con esto mostrado, se analiza --> Tipos de datos, Nulos, Duplicados
    print("Shape:")
    print(df.shape)
    print("Info:")
    print(df.info())
    print("Head:")
    print(df.head())

    # Eliminacion de duplicados:
    df = df.drop_duplicates()

    # Rellenar o eliminar nulos:
    df.dropna()

    # Estandarizacion de nombres de columnas:
    df.columns = df.columns.str.lower()

    # Carga de datos
    df.to_sql(table_name, con=engine, if_exists='append', index=False)


Shape:
(1000, 5)
Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Nombre      1000 non-null   object
 1   Apellido    1000 non-null   object
 2   DNI         1000 non-null   int64 
 3   Email       1000 non-null   object
 4   Contraseña  1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB
None
Head:
         Nombre   Apellido       DNI                             Email  \
0       Encarna    Donaire  49877134       encarna.donaire1@correo.com   
1  Jose Ignacio    Canales  98778810  jose ignacio.canales2@correo.com   
2          José  Sebastián  80095714        josé.sebastián3@correo.com   
3      Priscila      Vilar  94128750        priscila.vilar4@correo.com   
4      Epifanio       Ríos  62958342         epifanio.ríos5@correo.com   

      Contraseña  
0  Contraseña123  
1  Contraseña123  
2  Contraseña123  
3  Contra

2025-07-24 13:40:42,852 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-07-24 13:40:42,853 INFO sqlalchemy.engine.Engine [cached since 260.6s ago] {'table_name': 'usuarios', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-07-24 13:40:42,868 INFO sqlalchemy.engine.Engine INSERT INTO usuarios (nombre, apellido, dni, email, "contraseña") VALUES (%(nombre__0)s, %(apellido__0)s, %(dni__0)s, %(email__0)s, %(contraseña__0)s), (%(nombre__1)s, %(apellido__1)s, %(dni__1)s, %(email__1)s, %(contraseña__1)s), (%(nombre__2)s,